In [ ]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://fit.haui.edu.vn/vn"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
print(docs)

In [69]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))

model = genai.GenerativeModel('gemini-1.5-flash')

In [70]:
from langchain.chains import create_extraction_chain
from langchain_google_genai import ChatGoogleGenerativeAI

# os.environ['GOOGLE_API_KEY'] =os.environ["GEMINI_API_KEY"]
# print(os.environ.get('GOOGLE_API_KEY'))

# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}


def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=model).run(content)

In [71]:
from langchain_huggingface import HuggingFaceEmbeddings


model_name="dangvantuan/vietnamese-embedding"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddingsModel = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [72]:
import pprint

from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain_experimental.text_splitter import SemanticChunker

from langchain_huggingface import HuggingFaceEmbeddings


model_name="dangvantuan/vietnamese-embedding"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddingsModel = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

def scrape_with_playwright(urls, schema):
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=['span','p','li','']
    )

    text_splitter = SemanticChunker(embeddingsModel,breakpoint_threshold_type="gradient")
    
    splits = text_splitter.split_documents(docs_transformed)
    print(splits)

    # Process the first split
    extracted_content = extract(content=splits[0].page_content,schema=schema)
    pprint.pprint(extracted_content)
    return extracted_content



In [ ]:
schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}

# with open('link_result.txt','r') as link_result:
#     urls = link_result.readlines()
# print(urls)
urls = ["https://fit.haui.edu.vn/vn"]
extracted_content = scrape_with_playwright(urls, schema=schema)